<h1 style="text-align: center;">BCFind testing</h1>
This notebook gives an example on how to test BCFind by using modules and classes provided by the library


In [ ]:
import os
import lmdb
import json
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf

from bcfind.localizers import BlobDoG


gpus = tf.config.list_physical_devices("GPU")
tf.config.set_visible_devices(gpus[-1], "GPU")
tf.config.experimental.set_memory_growth(gpus[-1], True)

In [ ]:
path_to_my_data = "My_Data"
path_to_my_exp = "My_Exp"

max_input_shape = (160, 480, 480)
voxel_resolution = (2, 0.65, 0.65)
exclude_border = (3, 9, 9)
max_match_dist = 10

tiff_dir = f"{path_to_my_data}/Tiff_files/Test"
gt_dir = f"{path_to_my_data}/GT_files/Test"

tf_checkpoint_dir = f"{path_to_my_exp}/UNet_checkpoints"
dog_checkpoint_dir = f"{path_to_my_exp}/DoG_checkpoints"

lmdb_dir = f"{path_to_my_exp}/UNet_test_pred_lmdb"

In [ ]:
tiff_files = [f"{tiff_dir}/{fname}" for fname in os.listdir(tiff_dir)]
gt_files = [f"{gt_dir}/{fname}.marker" for fname in os.listdir(tiff_dir)]

In [ ]:
model = tf.keras.models.load_model(
    f"{tf_checkpoint_dir}/model.tf",
    compile=False,
)
model.build((None, None, None, None, 1))

In [ ]:
preprocessing = {
    "clip": "bit",
    "clip_value": 15,  # clip can be ['bit', 'constant', 'quantile', None]
    "center": None,
    "center_value": None,  # center can be one of ['constant', 'min', 'mean', null]
    "scale": "bit",
    "scale_value": 15,  # scale can be one of ['constant', 'bit', 'max', 'std', null]
}

In [ ]:
from bcfind.utils.models import predict
from bcfind.utils.data import get_input_tf, get_gt_as_numpy


n = len(tiff_files)
nbytes = np.prod(max_input_shape) * 1  # 4 bytes for float32: 1 byte for uint8
db = lmdb.open(lmdb_dir, map_size=n * nbytes * 10)

# UNet predictions
print(f"Saving U-Net predictions in {lmdb_dir}")
with db.begin(write=True) as fx:
    for i, tiff_file in enumerate(tiff_files):
        print(f"\nUnet prediction on file {i+1}/{len(tiff_files)}")

        x = get_input_tf(tiff_file, **preprocessing)
        pred = predict(x, model).numpy()
        pred = tf.sigmoid(tf.squeeze(pred)).numpy()
        pred = (pred * 255).astype("uint8")

        fname = tiff_file.split("/")[-1]
        fx.put(key=fname.encode(), value=pickle.dumps(pred))

db.close()
dog_inputs = lmdb.open(lmdb_dir, readonly=True)

# True cell coordinates
dog_targets = []
for marker_file in gt_files:
    print(f"Loading file {marker_file}")
    y = get_gt_as_numpy(marker_file)
    dog_targets.append(y)

In [ ]:
from bcfind.utils.base import evaluate_df


dog = BlobDoG(3, voxel_resolution, exclude_border)
dog_par = json.load(open(f"{dog_checkpoint_dir}/BlobDoG_parameters.json", "r"))
dog.set_parameters(dog_par)
print(f"Best parameters found for DoG: {dog_par}")

with dog_inputs.begin() as fx:
    db_iterator = fx.cursor()
    res = []
    for i, (fname, x) in enumerate(db_iterator):
        pred = dog.predict_and_evaluate(x, dog_targets[i], max_match_dist, "counts")

        pred["f1"] = pred["TP"] / (pred["TP"] + 0.5 * (pred["FP"] + pred["FN"]))
        pred["file"] = fname.decode()
        res.append(pred)
dog_inputs.close()

res = pd.concat(res)
res.to_csv(f"{path_to_my_exp}/Test_eval.csv", index=False)
perf = evaluate_df(res)

print(f"\nTest-set evaluated with {perf}")
print("")